# Retrieval augmented chat - fine tuned

This notebook is the primary demonstration of the project with the fine tuned model. Here we'll bring up the tuned model and vector database and start asking questions both with and without the vector database.

In [1]:
import chromadb
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from IPython.display import Markdown, display

## Initialize some variables

In [2]:
collection_name = "Corpus"
merged_model_dir = "merged-fine-tuned"
device_map = {"": 0}
device = "cuda"
database_top_n_results = 2

## Load the collection

In [3]:
client = chromadb.PersistentClient(path="db/")
collection = client.get_collection(name = collection_name)

## Load the model

In [4]:
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(merged_model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def send_to_model(msg):
    messages = [
        {"role": "user", "content": msg},
    ]
    
    encoded = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    generated_ids = merged_model.generate(encoded, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id, temperature=0.8)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded

def send_chat(msg):
    result = send_to_model(msg)[0]
    return result.rsplit(" [/INST] ", 1)[1]

def basic_chat(msg):
    print(send_chat(msg))

## Trying the model without access to the vector database.

In [6]:
basic_chat("How do I loop a GIF?")

To open a GIF in a loop in Python, use this code: 

```
import gifsicle

with open('file.gif', 'rb') as f:
    gif = gifsicle.GIF(f)

    for i in range(0, len(gif), 1):
        gif.display()
        gif.sleep()
```

This will open the GIF file in a loop, displaying each frame of the GIF for a set length of time determined by the `sleep()` method. The `range()` function specifies the number of times to loop through the file, starting at 0 and ending at the length of the GIF minus 1.

To make the loop indefinite, use this code:

```
import gifsicle

with open('file.gif', 'rb') as f:
    gif = gifsicle.GIF(f)

    while True:
        gif.display()
        gif.sleep()
```

This will keep the GIF looping indefinitely, displaying each frame until the program is closed.


## Add some retrieval convenience methods
These methods use the vector database to find the `database_top_n_results` from the vector database, add them into the request context, then annotate the result with links to the documents used in the context.

In [7]:
def printmd(string):
    display(Markdown(string))
    
def retrieval_augmented_chat(msg):
    query_result = collection.query(
        query_texts=[msg], 
        n_results=database_top_n_results
    )
    question_with_context = ""
    if len(query_result['documents'][0]) > 0:
        question_with_context = "Based on the following documents:\n" + "\n\n".join(query_result['documents'][0]) + "\n Answer the following question with lots of details: "
    question_with_context += msg
    model_response = send_chat(question_with_context)

    doc_links = ""
    if len(query_result['metadatas'][0]) > 0:
        doc_links = "\n\n **Reference documents:** \n\n"
        for i in range(0, len(query_result['metadatas'][0])):
            source = query_result['metadatas'][0][i]['source']
            score = query_result['distances'][0][i]
            doc_links += f"* [{source}]({source}) score: {score:3.2f}\n"
    return model_response + doc_links



## Run retrieval augmented chat
Notice that the responses have switched from a general HTML/JavaScript context to include the document contents which provide examples of encoding the gif with ImageMagick so that it loops by default.

In [8]:
printmd(retrieval_augmented_chat("How do I loop a GIF?"))

To make a GIF loop in ImageMagick, follow these steps:

1. Open a terminal window and navigate to the directory where the GIF file is saved.
2. Use the following command to open the GIF file in ImageMagick:
```
convert chrome-samesite-missing.gif
```
Replace `chrome-samesite-missing.gif` with the name of your GIF file.

3. Use the following command to make the GIF loop indefinitely:
```
-loop 0
```
This tells the GIF to loop indefinitely until the user stops it.

4. Use the following command to save the looping GIF:
```
chrome-samesite-missing-loop.gif
```
This saves the looping GIF with the same name as the original GIF, but with the word "loop" added to the end.

The complete command to loop a GIF in ImageMagick is:
```
convert chrome-samesite-missing.gif -loop 0 chrome-samesite-missing-loop.gif
```
Note that the output filename comes last, AFTER the `-loop 0` option.

 **Reference documents:** 

* [corpus/imagemagick/set-a-gif-to-loop.md](corpus/imagemagick/set-a-gif-to-loop.md) score: 0.97
* [corpus/imagemagick/compress-animated-gif.md](corpus/imagemagick/compress-animated-gif.md) score: 1.28


In [12]:
printmd(retrieval_augmented_chat("Can I use npx with GitHub actions?"))

Yes, you can use npx with GitHub actions. To do this, you will first need to install the required package globally using npm. For example, if you want to use the `npm install` command to install the required package globally, you can use the following code in your GitHub action workflow:
```yaml
- name: Install npm globally
  run: npm install -g npm
```
Once you have installed the required package globally, you can use the `npx` command in your workflow to execute the package. For example, if the package you want to use is called `my-package`, and it has a binary called `my-binary`, you can use the following code in your workflow:
```yaml
- name: Run my-binary using npx
  run: npx my-package/my-binary
```
This will install the required package using the globally installed npm, and then execute the `my-binary` binary from the package.

I hope this helps! Let me know if you have any other questions.

 **Reference documents:** 

* [corpus/github-actions/npm-cache-with-npx-no-package.md](corpus/github-actions/npm-cache-with-npx-no-package.md) score: 0.88
* [corpus/github-actions/attach-generated-file-to-release.md](corpus/github-actions/attach-generated-file-to-release.md) score: 1.11
